In [3]:
import os
import time
import logging 
from custome_logger import setup_custom_logger
logger = setup_custom_logger()
logger.debug("test")

import torch
import torch.nn as nn
import torch.utils.benchmark as benchmark
from timeit import default_timer as timer
import warnings
from mmcv import Config, DictAction
from mmcv.runner import get_dist_info, init_dist
from mmcv.runner import load_checkpoint, wrap_fp16_model
from os import path as osp
from mmdet3d.models import build_model
from mmdet3d.datasets import build_dataset
from mmcv.parallel import MMDataParallel
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)





In [1]:
from mmcv import Config

cfg = Config.fromfile(
    r"/home/niklas/ETM_BEV/BEVerse/projects/configs/beverse_tiny_exp.py"
)


In [2]:
cfg["model"]

{'type': 'BEVerse',
 'img_backbone': {'type': 'SwinTransformer',
  'pretrained': 'https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth',
  'pretrain_img_size': 224,
  'embed_dims': 96,
  'patch_size': 4,
  'window_size': 7,
  'mlp_ratio': 4,
  'depths': [2, 2, 6, 2],
  'num_heads': [3, 6, 12, 24],
  'strides': (4, 2, 2, 2),
  'out_indices': (2, 3),
  'qkv_bias': True,
  'qk_scale': None,
  'patch_norm': True,
  'drop_rate': 0.0,
  'attn_drop_rate': 0.0,
  'drop_path_rate': 0.0,
  'use_abs_pos_embed': False,
  'act_cfg': {'type': 'GELU'},
  'norm_cfg': {'type': 'LN', 'requires_grad': True},
  'pretrain_style': 'official',
  'output_missing_index_as_none': False},
 'img_neck': {'type': 'FPN_LSS', 'in_channels': 1152, 'inverse': True},
 'transformer': {'type': 'TransformerLSS',
  'grid_conf': {'xbound': [-51.2, 51.2, 0.2],
   'ybound': [-51.2, 51.2, 0.2],
   'zbound': [-10.0, 10.0, 20.0],
   'dbound': [1.0, 60.0, 0.5]},
  'input_dim': (256, 

In [5]:
def update_cfg(
    cfg,
    n_future=3,
    receptive_field=3,
    resize_lim=(0.38, 0.55),
    final_dim=(256, 704),
    grid_conf={
        "xbound": [-51.2, 51.2, 0.8],
        "ybound": [-51.2, 51.2, 0.8],
        "zbound": [-10.0, 10.0, 20.0],
        "dbound": [1.0, 60.0, 1.0],
    },
    det_grid_conf={
        "xbound": [-51.2, 51.2, 0.8],
        "ybound": [-51.2, 51.2, 0.8],
        "zbound": [-10.0, 10.0, 20.0],
        "dbound": [1.0, 60.0, 1.0],
    },
    map_grid_conf={
        "xbound": [-30.0, 30.0, 0.15],
        "ybound": [-15.0, 15.0, 0.15],
        "zbound": [-10.0, 10.0, 20.0],
        "dbound": [1.0, 60.0, 1.0],
    },
    motion_grid_conf={
        "xbound": [-50.0, 50.0, 0.5],
        "ybound": [-50.0, 50.0, 0.5],
        "zbound": [-10.0, 10.0, 20.0],
        "dbound": [1.0, 60.0, 1.0],
    },
    t_input_shape=(128, 128),
    point_cloud_range=[-51.2, -51.2, -5.0, 51.2, 51.2, 3.0],
):
    
    cfg["det_grid_conf"] = det_grid_conf
    cfg["map_grid_conf"] = map_grid_conf
    cfg["motion_grid_conf"] = motion_grid_conf
    cfg["grid_conf"] = det_grid_conf

    cfg["model"]["temporal_model"]["input_shape"] = t_input_shape

    cfg["data"]["val"]["pipeline"][0]["data_aug_conf"]["resize_lim"] = resize_lim
    cfg["data"]["train"]["dataset"]["pipeline"][0]["data_aug_conf"][
        "resize_lim"
    ] = resize_lim
    cfg["data"]["val"]["pipeline"][0]["data_aug_conf"]["final_dim"] = final_dim
    cfg["data"]["train"]["dataset"]["pipeline"][0]["data_aug_conf"][
        "final_dim"
    ] = final_dim

    cfg["data"]["test"]["pipeline"][0]["data_aug_conf"]["resize_lim"] = resize_lim
    cfg["data"]["test"]["pipeline"][0]["data_aug_conf"]["final_dim"] = final_dim

    cfg["model"]["pts_bbox_head"]["cfg_motion"][
        "grid_conf"
    ] = motion_grid_conf  # motion_grid
    cfg["model"]["temporal_model"]["grid_conf"] = grid_conf
    cfg["model"]["transformer"]["grid_conf"] = grid_conf
    cfg["model"]["pts_bbox_head"]["grid_conf"] = grid_conf
    cfg["data"]["train"]["dataset"]["grid_conf"] = motion_grid_conf
    cfg["data"]["val"]["pipeline"][3]["grid_conf"] = motion_grid_conf
    cfg["data"]["test"]["pipeline"][3]["grid_conf"] = motion_grid_conf
    cfg["data"]["val"]["grid_conf"] = grid_conf
    cfg["data"]["test"]["grid_conf"] = grid_conf

    cfg["model"]["pts_bbox_head"]["det_grid_conf"] = det_grid_conf

    cfg["model"]["pts_bbox_head"]["map_grid_conf"] = map_grid_conf
    cfg["data"]["train"]["dataset"]["map_grid_conf"] = map_grid_conf
    cfg["data"]["test"]["pipeline"][2]["map_grid_conf"] = map_grid_conf
    cfg["data"]["val"]["pipeline"][2]["map_grid_conf"] = map_grid_conf
    cfg["data"]["test"]["map_grid_conf"] = map_grid_conf
    cfg["data"]["val"]["map_grid_conf"] = map_grid_conf

    cfg["model"]["pts_bbox_head"]["motion_grid_conf"] = motion_grid_conf

    cfg["data"]["test"]["pipeline"][5]["point_cloud_range"] = point_cloud_range
    cfg["data"]["train"]["pipeline"][5][
        "point_cloud_range"
    ] = point_cloud_range  # point_cloud_range=None
    cfg["data"]["train"]["pipeline"][6][
        "point_cloud_range"
    ] = point_cloud_range  #'point_cloud_range =None
    cfg["data"]["val"]["pipeline"][5]["point_cloud_range"] = point_cloud_range

    cfg["model"]["pts_bbox_head"]["cfg_motion"]["receptive_field"] = receptive_field
    cfg["data"]["train"]["dataset"]["receptive_field"] = receptive_field
    cfg["model"]["temporal_model"]["receptive_field"] = receptive_field
    cfg["data"]["test"]["receptive_field"] = receptive_field
    cfg["data"]["val"]["receptive_field"] = receptive_field

    cfg["data"]["val"]["future_frames"] = n_future
    cfg["model"]["pts_bbox_head"]["cfg_motion"]["n_future"] = n_future
    cfg["data"]["test"]["future_frames"] = n_future
    cfg["data"]["train"]["dataset"]["future_frames"] = n_future
    
    cfg["data"]["train"]["dataset"]["pipeline"][4]["map_grid_conf"] = map_grid_conf
    cfg["data"]["train"]["dataset"]["pipeline"][5]["grid_conf"] = motion_grid_conf
    cfg["data"]["train"]["dataset"]["pipeline"][7]["point_cloud_range"] = point_cloud_range

    return cfg

def import_modules_load_config(cfg_file="beverse_tiny.py", samples_per_gpu=1):
    cfg_path = r"/home/niklas/ETM_BEV/BEVerse/projects/configs"
    cfg_path = os.path.join(cfg_path, cfg_file)

    cfg = Config.fromfile(cfg_path)

    # if args.cfg_options is not None:
    #     cfg.merge_from_dict(args.cfg_options)
    # import modules from string list.
    if cfg.get("custom_imports", None):
        from mmcv.utils import import_modules_from_strings

        import_modules_from_strings(**cfg["custom_imports"])

    # import modules from plguin/xx, registry will be updated
    if hasattr(cfg, "plugin"):
        if cfg.plugin:
            import importlib

            if hasattr(cfg, "plugin_dir"):
                plugin_dir = cfg.plugin_dir
                _module_dir = os.path.dirname(plugin_dir)
                _module_dir = _module_dir.split("/")
                _module_path = _module_dir[0]

                for m in _module_dir[1:]:
                    _module_path = _module_path + "." + m
                print(_module_path)
                plg_lib = importlib.import_module(_module_path)
            else:
                # import dir is the dirpath for the config file
                _module_dir = cfg_path
                _module_dir = _module_dir.split("/")
                _module_path = _module_dir[0]
                for m in _module_dir[1:]:
                    _module_path = _module_path + "." + m
                print(_module_path)
                plg_lib = importlib.import_module(_module_path)

    samples_per_gpu = 1
    if isinstance(cfg.data.test, dict):
        cfg.data.test.test_mode = True
        samples_per_gpu = cfg.data.test.pop("samples_per_gpu", 1)
        if samples_per_gpu > 1:
            # Replace 'ImageToTensor' to 'DefaultFormatBundle'
            cfg.data.test.pipeline = replace_ImageToTensor(cfg.data.test.pipeline)
    elif isinstance(cfg.data.test, list):
        for ds_cfg in cfg.data.test:
            ds_cfg.test_mode = True
        samples_per_gpu = max(
            [ds_cfg.pop("samples_per_gpu", 1) for ds_cfg in cfg.data.test]
        )
        if samples_per_gpu > 1:
            for ds_cfg in cfg.data.test:
                ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

    return cfg

torch.backends.cudnn.benchmark = True


@torch.no_grad()
def measure_time_host(
    model: nn.Module,
    input_tensor: torch.Tensor,
    num_repeats: int = 100,
    num_warmups: int = 10,
    synchronize: bool = True,
    continuous_measure: bool = True,
) -> float:

    for _ in range(num_warmups):
        _ = model.forward(input_tensor)
    torch.cuda.synchronize()

    elapsed_time_ms = 0

    if continuous_measure:
        start = timer()
        for _ in range(num_repeats):
            _ = model.forward(input_tensor)
        if synchronize:
            torch.cuda.synchronize()
        end = timer()
        elapsed_time_ms = (end - start) * 1000

    else:
        for _ in range(num_repeats):
            start = timer()
            _ = model.forward(input_tensor)
            if synchronize:
                torch.cuda.synchronize()
            end = timer()
            elapsed_time_ms += (end - start) * 1000

    return elapsed_time_ms / num_repeats


@torch.no_grad()
def measure_time_device(
    model: nn.Module,
    input_tensor: torch.Tensor,
    num_repeats: int = 100,
    num_warmups: int = 10,
    synchronize: bool = True,
    continuous_measure: bool = True,
) -> float:

    for _ in range(num_warmups):
        _ = model.forward(input_tensor)
    torch.cuda.synchronize()

    elapsed_time_ms = 0

    if continuous_measure:
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        start_event.record()
        for _ in range(num_repeats):
            _ = model.forward(input_tensor)
        end_event.record()
        if synchronize:
            # This has to be synchronized to compute the elapsed time.
            # Otherwise, there will be runtime error.
            torch.cuda.synchronize()
        elapsed_time_ms = start_event.elapsed_time(end_event)

    else:
        for _ in range(num_repeats):
            start_event = torch.cuda.Event(enable_timing=True)
            end_event = torch.cuda.Event(enable_timing=True)
            start_event.record()
            _ = model.forward(input_tensor)
            end_event.record()
            if synchronize:
                # This has to be synchronized to compute the elapsed time.
                # Otherwise, there will be runtime error.
                torch.cuda.synchronize()
            elapsed_time_ms += start_event.elapsed_time(end_event)

    return elapsed_time_ms / num_repeats


@torch.no_grad()
def run_inference(model: nn.Module, input_tensor: torch.Tensor) -> torch.Tensor:
    # from model.forward because BEVerse differentiates between different input types - img lidar etc 
    return model.forward_dummy()

def calculate_birds_eye_view_parameters(x_bounds, y_bounds, z_bounds):
    """
    Parameters
    ----------
        x_bounds: Forward direction in the ego-car.
        y_bounds: Sides
        z_bounds: Height

    Returns
    -------
        bev_resolution: Bird's-eye view bev_resolution
        bev_start_position Bird's-eye view first element
        bev_dimension Bird's-eye view tensor spatial dimension
    """
    bev_resolution = torch.tensor([row[2] for row in [x_bounds, y_bounds, z_bounds]])
    bev_start_position = torch.tensor(
        [row[0] + row[2] / 2.0 for row in [x_bounds, y_bounds, z_bounds]]
    )
    bev_dimension = torch.tensor(
        [(row[1] - row[0]) / row[2] for row in [x_bounds, y_bounds, z_bounds]],
        dtype=torch.long,
    )

    return bev_resolution, bev_start_position, bev_dimension

In [3]:
resize_lims = [
    (0.3, 0.45),  # fiery
    (0.38, 0.55),  # desTINY
    (0.82, 0.99),  # small
    (1, 1),  # BEVDEt
]

final_dims = [(224, 480), (256, 704), (512, 1408), (900, 1600)]

backbones = [
    "beverse_tiny.py",
    "beverse_tiny.py",
    "beverse_small.py",
    "beverse_small.py",
]

# future frames -> tiny settings
future_frames_list = [4, 4, 4, 4, 5, 7, 10]
receptive_field_list = [
    3,
    5,
    8,
    13,
    4,
    6,
    9,
]

# grid_size = (
#     point_cloud_range[3:] -  # type: ignore
#     point_cloud_range[:3]) / voxel_size  # type: ignore

point_cloud_range_base = [-51.2, -51.2, -5.0, 51.2, 51.2, 3.0]
point_cloud_range_extended_fustrum = [-71.2, -71.2, -5.0, 71.2, 71.2, 3.0]
det_grid_confs = {
    "xbound": [
        [-51.2, 51.2, 0.8],  # lower_bound, upper_bound, interval
        [-51.2, 51.2, 0.4],
        [-51.2, 51.2, 0.2],
        [-51.2, 51.2, 0.1],
        [-26.2, 26.2, 0.8],
        [-26.2, 26.2, 0.4],
    ],
    "ybound": [
        [-51.2, 51.2, 0.8],
        [-51.2, 51.2, 0.4],
        [-51.2, 51.2, 0.2],
        [-51.2, 51.2, 0.1],
        [-71.2, 71.2, 0.8],
        [-71.2, 71.2, 0.4],
    ],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [
        [1.0, 60.0, 1.0],
        [1.0, 60.0, 1.0],
        [1.0, 60.0, 0.5],
        [1.0, 60.0, 0.5],
        [1.0, 70.0, 1.0],
        [1.0, 70.0, 5.0],
    ],  # [(lower_bound, upper_bound, interval).]
}

motion_grid_confs = {
    "xbound": [
        [-50.0, 50.0, 0.5],
        [-50.0, 50.0, 0.25],
        [-50.0, 50.0, 0.125],
        [-50.0, 50.0, 0.075],
        [-25.0, 25.0, 0.5],
        [-25.0, 25.0, 0.25],
    ],
    "ybound": [
        [-50.0, 50.0, 0.5],
        [-50.0, 50.0, 0.25],
        [-50.0, 50.0, 0.125],
        [-50.0, 50.0, 0.075],
        [-70.0, 70.0, 0.5],
        [-70.0, 70.0, 0.25],
    ],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [
        [1.0, 60.0, 1.0],
        [1.0, 60.0, 1.0],
        [1.0, 60.0, 0.5],
        [1.0, 60.0, 0.5],
        [1.0, 70.0, 1.0],
        [1.0, 70.0, 5.0],
    ],
}

map_grid_confs = {
    "xbound": [-30.0, 30.0, 0.15],
    "ybound": [-15.0, 15.0, 0.15],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [
        [1.0, 60.0, 1.0],
        [1.0, 60.0, 1.0],
        [1.0, 60.0, 0.5],
        [1.0, 60.0, 0.5],
        [1.0, 70.0, 1.0],
        [1.0, 70.0, 5.0],
    ],
}
det_grid_conf = {
    "xbound": [-51.2, 51.2, 0.8],
    "ybound": [-51.2, 51.2, 0.8],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [1.0, 60.0, 1.0],
}

motion_grid_conf = {
    "xbound": [-50.0, 50.0, 0.5],
    "ybound": [-50.0, 50.0, 0.5],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [1.0, 60.0, 1.0],
}

map_grid_conf = {
    "xbound": [-30.0, 30.0, 0.15],
    "ybound": [-15.0, 15.0, 0.15],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [1.0, 60.0, 1.0],
}


In [27]:
# det_grid_conf["xbound"] = det_grid_confs["xbound"][1]
# det_grid_conf["ybound"] = det_grid_confs["ybound"][1]
# det_grid_conf["zbound"] = det_grid_confs["zbound"]
# det_grid_conf["dbound"] = det_grid_confs["dbound"][1]

# motion_grid_conf["xbound"] = motion_grid_confs["xbound"][1]
# motion_grid_conf["ybound"] = motion_grid_confs["ybound"][1]
# motion_grid_conf["zbound"] = motion_grid_confs["zbound"]
# motion_grid_conf["dbound"] = motion_grid_confs["dbound"][1]

# map_grid_conf["xbound"] = map_grid_confs["xbound"]
# map_grid_conf["ybound"] = map_grid_confs["ybound"]
# map_grid_conf["zbound"] = map_grid_confs["zbound"]
# map_grid_conf["dbound"] = map_grid_confs["dbound"][1]

In [4]:
# cfg = import_modules_load_config(cfg_file=backbones[0])
# cfg = update_cfg(
#     cfg,receptive_field=55,n_future=10
# )

projects.mmdet3d_plugin


In [13]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False,
)
iter_loader = iter(data_loader)
sample = next(iter_loader)

In [3]:

# device = torch.device("cuda:0")
# cfg = Config.fromfile(
#     r"/home/niklas/ETM_BEV/BEVerse/projects/configs/beverse_tiny.py"
# )

# if args.cfg_options is not None:
#     cfg.merge_from_dict(args.cfg_options)
# import modules from string list.
if cfg.get("custom_imports", None):
    from mmcv.utils import import_modules_from_strings

    import_modules_from_strings(**cfg["custom_imports"])

# import modules from plguin/xx, registry will be updated
if hasattr(cfg, "plugin"):
    if cfg.plugin:
        import importlib

        if hasattr(cfg, "plugin_dir"):
            plugin_dir = cfg.plugin_dir
            _module_dir = os.path.dirname(plugin_dir)
            _module_dir = _module_dir.split("/")
            _module_path = _module_dir[0]

            for m in _module_dir[1:]:
                _module_path = _module_path + "." + m
            print(_module_path)
            plg_lib = importlib.import_module(_module_path)
        else:
            # import dir is the dirpath for the config file
            _module_dir = os.path.dirname(
                r"/home/niklas/ETM_BEV/BEVerse/projects/configs/beverse_tiny_exp.py"
            )
            _module_dir = _module_dir.split("/")
            _module_path = _module_dir[0]
            for m in _module_dir[1:]:
                _module_path = _module_path + "." + m
            print(_module_path)
            plg_lib = importlib.import_module(_module_path)
            

samples_per_gpu = 1
if isinstance(cfg.data.test, dict):
    cfg.data.test.test_mode = True
    samples_per_gpu = cfg.data.test.pop("samples_per_gpu", 1)
    if samples_per_gpu > 1:
        # Replace 'ImageToTensor' to 'DefaultFormatBundle'
        cfg.data.test.pipeline = replace_ImageToTensor(cfg.data.test.pipeline)
elif isinstance(cfg.data.test, list):
    for ds_cfg in cfg.data.test:
        ds_cfg.test_mode = True
    samples_per_gpu = max(
        [ds_cfg.pop("samples_per_gpu", 1) for ds_cfg in cfg.data.test]
    )
    if samples_per_gpu > 1:
        for ds_cfg in cfg.data.test:
            ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

torch.backends.cudnn.benchmark = True


@torch.no_grad()
def measure_time_host(
    model: nn.Module,
    input_tensor: torch.Tensor,
    num_repeats: int = 100,
    num_warmups: int = 10,
    synchronize: bool = True,
    continuous_measure: bool = True,
) -> float:

    for _ in range(num_warmups):
        _ = model.forward(input_tensor)
    torch.cuda.synchronize()

    elapsed_time_ms = 0

    if continuous_measure:
        start = timer()
        for _ in range(num_repeats):
            _ = model.forward(input_tensor)
        if synchronize:
            torch.cuda.synchronize()
        end = timer()
        elapsed_time_ms = (end - start) * 1000

    else:
        for _ in range(num_repeats):
            start = timer()
            _ = model.forward(input_tensor)
            if synchronize:
                torch.cuda.synchronize()
            end = timer()
            elapsed_time_ms += (end - start) * 1000

    return elapsed_time_ms / num_repeats


@torch.no_grad()
def measure_time_device(
    model: nn.Module,
    input_tensor: torch.Tensor,
    num_repeats: int = 100,
    num_warmups: int = 10,
    synchronize: bool = True,
    continuous_measure: bool = True,
) -> float:

    for _ in range(num_warmups):
        _ = model.forward(input_tensor)
    torch.cuda.synchronize()

    elapsed_time_ms = 0

    if continuous_measure:
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        start_event.record()
        for _ in range(num_repeats):
            _ = model.forward(input_tensor)
        end_event.record()
        if synchronize:
            # This has to be synchronized to compute the elapsed time.
            # Otherwise, there will be runtime error.
            torch.cuda.synchronize()
        elapsed_time_ms = start_event.elapsed_time(end_event)

    else:
        for _ in range(num_repeats):
            start_event = torch.cuda.Event(enable_timing=True)
            end_event = torch.cuda.Event(enable_timing=True)
            start_event.record()
            _ = model.forward(input_tensor)
            end_event.record()
            if synchronize:
                # This has to be synchronized to compute the elapsed time.
                # Otherwise, there will be runtime error.
                torch.cuda.synchronize()
            elapsed_time_ms += start_event.elapsed_time(end_event)

    return elapsed_time_ms / num_repeats


@torch.no_grad()
def run_inference(model: nn.Module, input_tensor: torch.Tensor) -> torch.Tensor:

    return model.forward(input_tensor)



projects.mmdet3d_plugin


In [ ]:
det_grid_conf = {
    "xbound": [-62.0, 62.0, 0.8],
    "ybound": [-36.2, 36.2, 0.8],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [1.0, 70.0, 1.0],
}

motion_grid_conf = {
    "xbound": [-60.0, 60.0, 0.5],
    "ybound": [-36.0, 36.0, 0.5],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [1.0, 70.0, 1.0],
}

map_grid_conf = {
    "xbound": [-30.0, 30.0, 0.15],
    "ybound": [-15.0, 15.0, 0.15],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [1.0, 70.0, 1.0],
}

----------
det_grid_conf = {
    "xbound": [-51.2, 51.2, 0.8],
    "ybound": [-51.2, 51.2, 0.8],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [1.0, 60.0, 1.0],
}

motion_grid_conf = {
    "xbound": [-50.0, 50.0, 0.5],
    "ybound": [-50.0, 50.0, 0.5],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [1.0, 60.0, 1.0],
}

map_grid_conf = {
    "xbound": [-30.0, 30.0, 0.15],
    "ybound": [-15.0, 15.0, 0.15],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [1.0, 60.0, 1.0],
}

In [10]:

x_bounds, y_bounds,z_bounds=cfg["det_grid_conf"]["xbound"],cfg["det_grid_conf"]["ybound"],cfg["det_grid_conf"]["zbound"]

bev_dimension = torch.tensor(
        [(row[1] - row[0]) / row[2] for row in [x_bounds, y_bounds, z_bounds]],
        dtype=torch.long,
    )

In [17]:
for row in [x_bounds, y_bounds, z_bounds]:
    print((row[1] - row[0]) / row[2])

155.0
90.5
1.0


In [ ]:
# det_grid_conf = {
#     "xbound": [-62.0, 62.0, 0.37],#37
#     "ybound": [-36.2, 36.2, 0.375],#37,5
#     "zbound": [-10.0, 10.0, 20.0],
#     "dbound": [1.0, 70.0, 1.0],
# }

# motion_grid_conf = {
#     "xbound": [-60.0, 60.0, 0.25],
#     "ybound": [-36.0, 36.0, 0.25],
#     "zbound": [-10.0, 10.0, 20.0],
#     "dbound": [1.0, 70.0, 1.0],
# }

# map_grid_conf = {
#     "xbound": [-30.0, 30.0, 0.15],
#     "ybound": [-15.0, 15.0, 0.15],
#     "zbound": [-10.0, 10.0, 20.0],
#     "dbound": [1.0, 70.0, 1.0],
# }

In [6]:
det_grid_conf = {
    "xbound": [-62.0, 62.0, 0.37],#37
    "ybound": [-36.2, 36.2, 0.375],#37,5
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [1.0, 70.0, 1.0],
}

motion_grid_conf = {
    "xbound": [-60.0, 60.0, 0.25],
    "ybound": [-36.0, 36.0, 0.25],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [1.0, 70.0, 1.0],
}

map_grid_conf = {
    "xbound": [-30.0, 30.0, 0.15],
    "ybound": [-15.0, 15.0, 0.15],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [1.0, 70.0, 1.0],
}

point_cloud_range_base = [-51.2, -51.2, -5.0, 51.2, 51.2, 3.0]
point_cloud_range_extended_fustrum = [-62.0, -62.0, -5.0, 62.0, 62.0, 3.0]


cfg = import_modules_load_config(cfg_file="/home/niklas/ETM_BEV/BEVerse/projects/configs/beverse_tiny_exp.py")
cfg = update_cfg(
    cfg,det_grid_conf=det_grid_conf,grid_conf=det_grid_conf, map_grid_conf=map_grid_conf, motion_grid_conf=motion_grid_conf, point_cloud_range=point_cloud_range_base
)

projects.mmdet3d_plugin


In [7]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False)

sample = next(iter(data_loader))

model = build_model(cfg.model, test_cfg=cfg.get("test_cfg"))
wrap_fp16_model(model)
model.cuda()
model = MMDataParallel(model, device_ids=[0])


Upsample(scale_factor=4.0, mode=bilinear)
Upsample(scale_factor=4.0, mode=bilinear)
Upsample(scale_factor=4.0, mode=bilinear)
CenterHeadv1
CenterPointBBoxCoder
MapHead
BaseMotionHead
IterativeFlow
Temp3d:  (128, 128)
Temp3d:  {'xbound': [-62.0, 62.0, 0.37], 'ybound': [-36.2, 36.2, 0.375], 'zbound': [-10.0, 10.0, 20.0], 'dbound': [1.0, 70.0, 1.0]}
pool_sizes [(2, 96, 167)]
temp block, in_channels  70 reduction_channels  23 pool_sizes  [(2, 96, 167)] 
Temproal3DConvModel: block_in_channels 70, block_out_channels 64, pool_sizes [(2, 96, 167)]
pool_sizes [(2, 96, 167)]
temp block, in_channels  64 reduction_channels  21 pool_sizes  [(2, 96, 167)] 
Temproal3DConvModel: block_in_channels 64, block_out_channels 64, pool_sizes [(2, 96, 167)]


In [9]:
logger.debug(cfg["det_grid_conf"])
logger.debug(cfg["motion_grid_conf"])
logger.debug(cfg["map_grid_conf"])


bev_resolution, bev_start_position, bev_dimension=calculate_birds_eye_view_parameters(cfg["det_grid_conf"]["xbound"],cfg["det_grid_conf"]["ybound"],cfg["det_grid_conf"]["zbound"])
logger.debug(f"bev_resolution: {bev_resolution}")
logger.debug(f"bev_start_position: {bev_start_position}")
logger.debug(f"bev_dimension: {bev_dimension}")


motion_distribution_targets = {
    # for motion prediction
    'motion_segmentation': sample['motion_segmentation'][0],
    'motion_instance': sample['motion_instance'][0],
    'instance_centerness': sample['instance_centerness'][0],
    'instance_offset': sample['instance_offset'][0],
    'instance_flow': sample['instance_flow'][0],
    'future_egomotion': sample['future_egomotions'][0],
}


with torch.no_grad():
    result = model(
        return_loss=False,
        rescale=True,
        img_metas=sample['img_metas'],
        img_inputs=sample['img_inputs'],
        future_egomotions=sample['future_egomotions'],
        motion_targets=motion_distribution_targets,
        img_is_valid=sample['img_is_valid'][0],
    )


update
pyramid pooling:  torch.Size([1, 70, 3, 96, 167])
x_pool:  torch.Size([1, 23, 3, 1, 1])
c:  23
x_pool2:  torch.Size([3, 23, 96, 167])
x_pool3:  torch.Size([1, 23, 3, 96, 167])
update
pyramid pooling:  torch.Size([1, 64, 3, 96, 167])
x_pool:  torch.Size([1, 21, 3, 1, 1])
c:  21
x_pool2:  torch.Size([3, 21, 96, 167])
x_pool3:  torch.Size([1, 21, 3, 96, 167])
feats-1: torch.Size([1, 512, 25, 50]), feats-3: torch.Size([1, 128, 100, 200])
Up: x1 torch.Size([1, 512, 100, 200]), x2 torch.Size([1, 128, 100, 200])
feats-1: torch.Size([1, 512, 12, 21]), feats-3: torch.Size([1, 128, 48, 84])
Up: x1 torch.Size([1, 512, 48, 84]), x2 torch.Size([1, 128, 48, 84])
feats-1: torch.Size([1, 512, 36, 60]), feats-3: torch.Size([1, 128, 144, 240])
Up: x1 torch.Size([1, 512, 144, 240]), x2 torch.Size([1, 128, 144, 240])


In [12]:
image (T: 3 H: 65 W: 178) smaller than kernel size (kT: 2 kH: 128 kW: 128)


update
pyramid pooling:  torch.Size([1, 70, 3, 128, 128])
x_pool:  torch.Size([1, 23, 3, 1, 1])
c:  23
x_pool2:  torch.Size([3, 23, 128, 128])
x_pool3:  torch.Size([1, 23, 3, 128, 128])
update
pyramid pooling:  torch.Size([1, 64, 3, 128, 128])
x_pool:  torch.Size([1, 21, 3, 1, 1])
c:  21
x_pool2:  torch.Size([3, 21, 128, 128])
x_pool3:  torch.Size([1, 21, 3, 128, 128])
feats-1: torch.Size([1, 512, 25, 50]), feats-3: torch.Size([1, 128, 100, 200])
Up: x1 torch.Size([1, 512, 100, 200]), x2 torch.Size([1, 128, 100, 200])
feats-1: torch.Size([1, 512, 16, 16]), feats-3: torch.Size([1, 128, 64, 64])
Up: x1 torch.Size([1, 512, 64, 64]), x2 torch.Size([1, 128, 64, 64])
feats-1: torch.Size([1, 512, 25, 25]), feats-3: torch.Size([1, 128, 100, 100])
Up: x1 torch.Size([1, 512, 100, 100]), x2 torch.Size([1, 128, 100, 100])

SyntaxError: invalid syntax (2716849234.py, line 1)

In [10]:
config = {"IMAGE": {
    "ORIGINAL_HEIGHT": 900,
    "ORIGINAL_WIDTH": 1600,
    "FINAL_DIM": (512, 1408),
    "RESIZE_SCALE": 0.25,
    "TOP_CROP": 0,
}}

In [11]:
config.keys()

dict_keys(['IMAGE'])

In [49]:


def get_resizing_and_cropping_parameters(config):
    original_height, original_width = config["IMAGE"]["ORIGINAL_HEIGHT"], config["IMAGE"]["ORIGINAL_WIDTH"]
    final_height, final_width = config["IMAGE"]["FINAL_DIM"]

    resize_scale = config["IMAGE"]["RESIZE_SCALE"]
    resize_dims = (int(original_width * resize_scale), int(original_height * resize_scale))
    resized_width, resized_height = resize_dims

    crop_h = config["IMAGE"]["TOP_CROP"]
    crop_w = int(max(0, (resized_width - final_width) / 2))
    # Left, top, right, bottom crops.
    crop = (crop_w, crop_h, crop_w + final_width, crop_h + final_height)

    if resized_width != final_width:
        print('Zero padding left and right parts of the image.')
    if crop_h + final_height != resized_height:
        print('Zero padding bottom part of the image.')

    return {'scale_width': resize_scale,
            'scale_height': resize_scale,
            'resize_dims': resize_dims,
            'crop': crop,
            }
get_resizing_and_cropping_parameters(config)

Zero padding left and right parts of the image.
Zero padding bottom part of the image.


{'scale_width': 0.25,
 'scale_height': 0.25,
 'resize_dims': (400, 225),
 'crop': (0, 0, 1408, 512)}

In [20]:
result["time_stats"]

{'t_BEV': 1663055717.0110373,
 't_temporal': 1663055717.5791538,
 't0': 1663055712.6830127,
 't_end': 1663055746.884341}

x, rots, trans, intrins, post_rots, post_trans = input

'CAM_FRONT_RIGHT': {
    'data_path': './data/nuscenes/samples/CAM_FRONT_RIGHT/n015-2018-10-08-15-36-50+0800__CAM_FRONT_RIGHT__1538984233520339.jpg', 
    'type': 'CAM_FRONT_RIGHT', 
    'sample_data_token': 'd6f89460954c43d39ed7c9ac91ab03d0', 
    'sensor2ego_translation': [1.5508477543, -0.493404796419, 1.49574800619], 
    'sensor2ego_rotation': [0.2060347966337182, -0.2026940577919598, 0.6824507824531167, -0.6713610884174485], 'ego2global_translation': [715.6566537856895, 1810.1516804263824, 0.0], 
    'ego2global_rotation': [0.8004835927391405, 0.00541081062084495, -0.0028680900818508943, -0.5993233809414961], 'timestamp': 1538984233520339, 
    'sensor2lidar_rotation': array([[ 0.55971752, -0.01057234,  0.82861603],
       [-0.82828535,  0.02385392,  0.55979851],
       [-0.02568412, -0.99965955,  0.00459454]]), 
    'sensor2lidar_translation': array([ 0.48135698,  0.51094805, -0.32997566]), 
    'cam_intrinsic': array([[1.26084744e+03, 0.00000000e+00, 8.07968245e+02],
       [0.00000000e+00, 1.26084744e+03, 4.95334427e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])},

In [6]:
batch_size = 1
num_warmups = 100
num_repeats = 1000
# Change to C x 1 x 3 x 1600 x 900
# 704×256 Tiny
# 1408×512
# B, S, N, C, imH, imW = imgs.shape
img_inputs =  torch.rand(1,7,6,3,704,256).cuda()
# (batch, seq, num_cam)
future_egomotions = torch.zeros((batch_size, 7, 6)).type_as(img_inputs).cuda()
img_is_valid = torch.ones((batch_size, 7)).type_as(img_inputs) > 0
img_is_valid.cuda()
print("Future Egomotions: ", future_egomotions.shape)
print("img_is_valid: ", img_is_valid.shape)


Future Egomotions:  torch.Size([1, 7, 6])
img_is_valid:  torch.Size([1, 7])


In [7]:
# #single frame 
# future_egomotions = future_egomotions[:, :1]
# img_is_valid = img_is_valid[:, :1]
# print("Future Egomotions: ", future_egomotions.shape)
# print("img_is_valid: ", img_is_valid.shape)


In [8]:
from mmdet3d.core.bbox.structures.box_3d_mode import LiDARInstance3DBoxes

dummy_lidar2ego_rots = (
    torch.tensor(
        [
            [-5.4280e-04, 9.9893e-01, 4.6229e-02],
            [-1.0000e00, -4.0569e-04, -2.9750e-03],
            [-2.9531e-03, -4.6231e-02, 9.9893e-01],
        ]
    )
    .type_as(img_inputs)
    .cpu()
)
dummy_lidar2ego_trans = (
    torch.tensor([0.9858, 0.0000, 1.8402]).type_as(img_inputs).cpu()
)
print(dummy_lidar2ego_rots.shape)
print(dummy_lidar2ego_trans.shape)
img_metas = [
    dict(
        box_type_3d=LiDARInstance3DBoxes,
        lidar2ego_rots=dummy_lidar2ego_rots,
        lidar2ego_trans=dummy_lidar2ego_trans,
    )
]
print("image meta:", img_metas)

torch.Size([3, 3])
torch.Size([3])
image meta: [{'box_type_3d': <class 'mmdet3d.core.bbox.structures.lidar_box3d.LiDARInstance3DBoxes'>, 'lidar2ego_rots': tensor([[-5.4280e-04,  9.9893e-01,  4.6229e-02],
        [-1.0000e+00, -4.0569e-04, -2.9750e-03],
        [-2.9531e-03, -4.6231e-02,  9.9893e-01]]), 'lidar2ego_trans': tensor([0.9858, 0.0000, 1.8402])}]


In [20]:
model = build_model(cfg.model, test_cfg=cfg.get("test_cfg"))
print(type(model))


<class 'projects.mmdet3d_plugin.models.detectors.beverse.BEVerse'>


In [9]:
model = build_model(cfg.model, test_cfg=cfg.get("test_cfg")).cuda()
wrap_fp16_model(model)
img_feats,time_dict  = model.extract_img_feat(
            img=img_inputs,
            img_metas=img_metas,
            future_egomotion=future_egomotions,
            img_is_valid=img_is_valid,
        )
print(img_feats.shape)
print(time_dict)

batch?:  torch.Size([1, 7, 6, 3, 704, 256])
B 1, S 7, 6, C 3, imH 704, imW 256
imgs  torch.Size([42, 3, 704, 256])
after backbone:  2
shape in list:  [torch.Size([42, 384, 44, 16]), torch.Size([42, 768, 22, 8])]
after backbone with_img_neck:  torch.Size([42, 512, 44, 16])
after transformation:  torch.Size([1, 7, 6, 512, 44, 16])


TypeError: can only concatenate list (not "Tensor") to list

In [1]:
import torch



In [6]:
# 'xbound': [-51.2, 51.2, 0.2], 'ybound': [-51.2, 51.2, 0.2], 'zbound': [-10.0, 10.0, 20.0], 'dbound': [1.0, 60.0, 0.5]}



x_bounds, y_bounds, z_bounds = [-51.2, 51.2, 0.2], [-51.2, 51.2, 0.2], [-10.0, 10.0, 20.0]

bev_resolution, bev_start_position, bev_dimension = calculate_birds_eye_view_parameters(x_bounds, y_bounds, z_bounds)

print(bev_resolution, bev_start_position, bev_dimension)

tensor([ 0.2000,  0.2000, 20.0000]) tensor([-51.1000, -51.1000,   0.0000]) tensor([512, 512,   1])


/tmp/ipykernel_362/3561629359.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  bev_dimension = torch.tensor(


In [ ]:

x_bounds, y_bounds, z_bounds = [-26.2, 26.2, 0.8], [-71.2, 71.2, 0.8], [-10.0, 10.0, 20.0]
tensor([ 0.8000,  0.8000, 20.0000]) tensor([-25.8000, -70.8000,   0.0000]) tensor([ 65, 178,   1])


x_bounds, y_bounds, z_bounds = [-26.2, 26.2, 0.4], [-71.2, 71.2, 0.4], [-10.0, 10.0, 20.0]
tensor([ 0.4000,  0.4000, 20.0000]) tensor([-26., -71.,   0.]) tensor([131, 356,   1])


In [13]:
img_inputs =  torch.rand(1,7,6,3,704,256)
print(img_inputs[1:].shape)
trans_output = torch.rand(1, 7, 6, 512, 44, 16)
img_inputs =  torch.rand(2,7,6,3,704,256)
print(img_inputs[1:].shape)
test = [trans_output] + img_inputs[1:]
test.shape

torch.Size([0, 7, 6, 3, 704, 256])
torch.Size([1, 7, 6, 3, 704, 256])


TypeError: can only concatenate list (not "Tensor") to list

In [16]:

num_warmups = 100
num_repeats = 1000
# Change to C x 1 x 3 x 1600 x 900
# 704×256 Tiny
# 1408×512
# B, T, N, C, imH, imW = imgs.shape
input_shape = (1,3,6,3,1600,900)

future_egomotions = torch.zeros((batch_size, 7, 6)).type_as(img_inputs)
img_is_valid = torch.ones((batch_size, 7)).type_as(img_inputs) > 0
print("Future Egomotions: ", future_egomotions.shape)
print("img_is_valid: ", img_is_valid.shape)


# imgs = imgs.view(B * S * N, C, imH, imW)

model = build_model(cfg.model, test_cfg=cfg.get("test_cfg"))
wrap_fp16_model(model)
# load_checkpoint(
#     model,
#     r"/home/niklas/ETM_BEV/BEVerse/checkpoints/beverse_tiny.pth",
#     map_location="cpu",
# )
# model = fuse_module(model)
model.cuda(device)
model.eval()
# model = nn.Conv2d(in_channels=input_shape[1], out_channels=256, kernel_size=(5, 5))

# Input tensor
input_tensor = torch.rand(input_shape, device=device)

torch.cuda.synchronize()

print("Latency Measurement Using CPU Timer...")
for continuous_measure in [True]:
    for synchronize in [True]:
        try:
            latency_ms = measure_time_host(
                model=model,
                input_tensor=input_tensor,
                num_repeats=num_repeats,
                num_warmups=num_warmups,
                synchronize=synchronize,
                continuous_measure=continuous_measure,
            )
            print(
                f"|"
                f"Synchronization: {synchronize!s:5}| "
                f"Continuous Measurement: {continuous_measure!s:5}| "
                f"Latency: {latency_ms:.5f} ms| "
            )
        except Exception as e:
            print(
                f"|"
                f"Synchronization: {synchronize!s:5}| "
                f"Continuous Measurement: {continuous_measure!s:5}| "
                f"Latency: N/A     ms| "
            )
        torch.cuda.synchronize()

print("Latency Measurement Using CUDA Timer...")
for continuous_measure in [True, False]:
    for synchronize in [True, False]:
        try:
            latency_ms = measure_time_device(
                model=model,
                input_tensor=input_tensor,
                num_repeats=num_repeats,
                num_warmups=num_warmups,
                synchronize=synchronize,
                continuous_measure=continuous_measure,
            )
            print(
                f"|"
                f"Synchronization: {synchronize!s:5}| "
                f"Continuous Measurement: {continuous_measure!s:5}| "
                f"Latency: {latency_ms:.5f} ms| "
            )
        except Exception as e:
            print(
                f"|"
                f"Synchronization: {synchronize!s:5}| "
                f"Continuous Measurement: {continuous_measure!s:5}| "
                f"Latency: N/A     ms| "
            )
        torch.cuda.synchronize()

print("Latency Measurement Using PyTorch Benchmark...")
num_threads = 1
timer = benchmark.Timer(
    stmt="run_inference(model, input_tensor)",
    setup="from __main__ import run_inference",
    globals={"model": model, "input_tensor": input_tensor},
    num_threads=num_threads,
    label="Latency Measurement",
    sub_label="torch.utils.benchmark.",
)

profile_result = timer.timeit(num_repeats)
# https://pytorch.org/docs/stable/_modules/torch/utils/benchmark/utils/common.html#Measurement
print(f"Latency: {profile_result.mean * 1000:.5f} ms")


Latency Measurement Using CPU Timer...
|Synchronization: True | Continuous Measurement: True | Latency: N/A     ms| 
Latency Measurement Using CUDA Timer...
|Synchronization: True | Continuous Measurement: True | Latency: N/A     ms| 
|Synchronization: False| Continuous Measurement: True | Latency: N/A     ms| 
|Synchronization: True | Continuous Measurement: False| Latency: N/A     ms| 
|Synchronization: False| Continuous Measurement: False| Latency: N/A     ms| 
Latency Measurement Using PyTorch Benchmark...


FileNotFoundError: [Errno 2] No such file or directory: 'dummy_input.pt'